In [ ]:
!pip install selenium webdriver-manager beautifulsoup4 pdfplumber pandas matplotlib 

In [ ]:
import re
import os
import time

import pdfplumber # доставать из pdf данные

import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup 

# Сделать датасет с данными по годам о выручке netflix
1. Скачать со страницы netflix финансовые отчеты в `pdf`
2. Получить с `pdf` данные для таблиц
3. Преобразовать в таблицу и сохранить датасет

In [ ]:
service = webdriver.ChromeService(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chrome.exe') 
                                  

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
            "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
browser = webdriver.Chrome(options=options)
link = 'https://ir.netflix.net/financials/annual-reports-and-proxies/default.aspx'
browser.get(link)

In [ ]:
folder = 'annual_reports/'

### 1. Получить все ссылки ведущие на отчеты и автоматизированно их скачать

Должен быть включен VPN.

In [ ]:
# страница `https://ir.netflix.net/financials/annual-reports-and-proxies/default.aspx` была заранее скачана
with open(f'{folder}Netflix - Financials - Annual Reports & Proxies.html') as fp:
    soup = BeautifulSoup(fp)

In [ ]:
tags = soup.find_all('a', {'class':'module_link module-financial_link'})
tags[0]

In [ ]:
type(tags[0])

In [ ]:
tags[0]['href']

In [ ]:
len(tags)

In [ ]:
lst = []
for tag in tags:
    for child in tag.children:
        if 'Annual Report' in child.text:
            print(child.text)
            lst.append(tag['href'])
            
print(len(lst))

In [ ]:
lst

In [ ]:
# скачивание pdf файлов
for link in lst:
    browser.get(link)
    

## анализ pdf файлов

1. Необходимо получить данные для таблиц `CONSOLIDATED STATEMENTS OF OPERATIONS`

In [ ]:
# из папки Downloads файлы были перемещены в папку с отчетами
reports = [f for f in os.listdir(folder) if f.endswith('pdf')]
print(len(reports))
reports[:2]

In [ ]:
folder_text = 'annual_reports_parsed/'

In [ ]:
for filename in reports:
    with pdfplumber.open(folder + filename) as pdf:
        print(filename, len(pdf.pages))
        for i, page in enumerate(pdf.pages):
            
            # print(page.chars[0])
            tables = page.extract_tables({'vertical_strategy':'text'})
            text = [x['text'] for x in page.extract_text_lines()]
            if 'CONSOLIDATED STATEMENTS OF OPERATIONS' in text:
                print('page', i)
                line = '\n'.join(text)
                with open(folder_text + filename.replace('.pdf', '.txt'), 'w') as file:
                    file.write(line)
                    
            elif 'CONSOLIDATEDSTATEMENTSOFOPERATIONS' in text:
                print('page', i)
                line = '\n'.join(text)
                with open(folder_text + filename.replace('.pdf', '.not_spaced.txt'), 'w') as file:
                    file.write(line)
 

In [ ]:
def preprocess_not_spaced(filename):
    print(filename)
    with open(filename, 'r') as file:
        lines = file.readlines()
    # print(lines)
    years = list(re.findall('([0-9]{4})', lines[4]))
    
    print(years)
    i = 5
    lst = ['column ' + lines[4].strip()]
    head = ''
    while i < len(lines):
        line = lines[i].strip()
        # print(line)
        if '..' not in line:
            head = line.replace(',', '').replace('$', '').replace(')', '').replace('(', '').replace('* ', '').replace('*', '').replace('—', '')
        i += 1
        while i < len(lines) and '..' in line:
            line = line.replace(',', '').replace('$', '').replace(')', '').replace('(', '').replace('* ', '').replace('*', '').replace('—', '')
            
            processed = re.sub('\.{2,}', '', line)
            lst += [head + processed]
            i += 1
            line = lines[i].strip()

    return lst

def parse_not_spaced(filename):
    lst = preprocess_not_spaced(filename)
    folder = 'annual_reports_parsed_step2/'
    processed_filename = folder + filename.split('/')[-1]
    with open(processed_filename, 'w') as file:
        for line in lst:
            file.write(line.replace('  ', ' ').replace('  ', ' ') + '\n')

    df = pd.read_csv(processed_filename, sep=' ')
    df = df.set_index('column').T.rename(columns={'Revenues:Subscription':'Revenues'})
    return df

        

In [ ]:
files_not_spaced = [f for f in os.listdir(folder_text) if 'not_spaced' in f]
files_spaced = [f for f in os.listdir(folder_text) if 'not_spaced' not in f]
print(len(files_not_spaced), len(files_spaced))
files_not_spaced[:2]

In [ ]:
lst = []
print(len(files_not_spaced))
for i in range(len(files_not_spaced)):
    print(i)
    try:
        lst.append(parse_not_spaced(folder_text + files_not_spaced[i]))
    except BaseException as e:
        print(files_not_spaced[i])
        print(e)

In [ ]:
merged = pd.concat(lst).reset_index().sort_values('index').rename(columns={'index':'year'})

In [ ]:
first = merged.groupby('year').first()
first

In [ ]:
def preprocess_spaced(filename):
    print(filename)
    with open(filename, 'r') as file:
        lines = file.readlines()[:-1] # 

    years = list(re.findall('([0-9]{4})', lines[5]))
    
    print(years)
    i = 6
    lst = ['\t'.join(['column'] + years)]
    head = ''
        
    while i < len(lines):      
        line = lines[i].replace(',', '').replace('$', '').replace(')', '').replace('(', '').replace('* ', '').replace('*', '').replace('—', '').strip()
        # print(i, line)
        if ':' in line:
            head = line
            i += 1
            line = lines[i].replace(',', '').replace('$', '').replace(')', '').replace('(', '').replace('* ', '').replace('*', '').replace('—', '').strip()
            
        name = re.findall('([a-zA-Z ]*)\d*.*', line)
        # print(name)
        numbers = [str(num) for num in re.findall(r'[\d]*[.]*[\d]+', line)]
        lst += [head + ''.join(name).strip() + '\t' + '\t'.join(numbers)]
        i += 1

    return lst

def parse_spaced(filename):
    lst = preprocess_spaced(filename)
    folder = 'annual_reports_parsed_step2/'
    processed_filename = folder + filename.split('/')[-1]
    print(processed_filename)
    with open(processed_filename, 'w') as file:
        for line in lst:
            file.write(line.replace('  ', ' ').replace('  ', ' ') + '\n')

    df = pd.read_csv(processed_filename, sep='\t')
    df = df.set_index('column').T.rename(columns={'Revenues:Subscription':'Revenues'})
    return df

        

In [ ]:
lst = []
print(len(files_spaced))
for i in range(len(files_spaced)):
    print(i)
    try:
        lst.append(parse_spaced(folder_text + files_spaced[i]))
    except BaseException as e:
        print(files_spaced[i])
        print(e)
    # break

In [ ]:
print(len(lst))
lst[0]

In [ ]:
merged = pd.concat(lst).reset_index().sort_values('index').rename(columns={'index':'year'})

In [ ]:
second = merged.groupby('year').first().rename(columns={'Other income expense:Net income':'Net_income'})
second

In [ ]:
cols = ['Revenues', 'Cost of revenues', 'Marketing',
       'Technology and development', 'General and administrative',
       'Operating income', 'Other income expense:Interest expense',
       'Other income expense:Interest and other income expense',
       'Other income expense:Income before income taxes',
       'Net_income', 'Earnings per share:Basic', 'Earnings per share:Diluted',
       'Weighted-average common shares outstanding:Basic',
       'Weighted-average common shares outstanding:Diluted',
       ]

In [ ]:
second = second[cols]

In [ ]:
rename = {x: x.lower().replace(' ', '_').replace('other_income_expense:', '')\
          .replace('earnings_per_share:', 'eps_').replace('weighted-average_common_shares_outstanding:', 'wacso_')
          for x in second.columns}
rename

In [ ]:
second = second.rename(columns=rename)

In [ ]:
assert second.shape[0] == second.index.nunique()

In [ ]:
second.info()

In [ ]:
rename_first = {'Revenues':'revenues', 'Marketing':'marketing', 
                'Technologyanddevelopment':'technology_and_development',
       'Generalandadministrative':'general_and_administrative',
                'Operatingincome':'operating_income',
       'Otherincomeexpense:Interestexpense':'interest_expense',
       'Otherincomeexpense:Incomebeforeincometaxes':'income_before_income_taxes',
       'Otherincomeexpense:Provisionforincometaxes':'provision_for_income_taxes',
       'Otherincomeexpense:Netincome':'ie_net_income', 
                'Earningspershare:Basic':'eps_basic',
       'Weighted-averagecommonsharesoutstanding:Basic':'wacso_basic', 
                'Subscription':'subscription',
       'Fulfillmentexpenses':'fulfillment_expenses', 
                'Totalcostofrevenues':'cost_of_revenues', 
                'Grossprofit':'grossprofit',
       'Operatingexpenses:Technologyanddevelopment':'oe_technology_and_development',
       'Operatingexpenses:Generalandadministrative':'oe_general_and_administrative',
       'Operatingexpenses:GainondisposalofDVDs':'oe_gain_on_disposal_of_DVDs',
       'Operatingexpenses:Totaloperatingexpenses':'oe_total_operating_expenses',
       'Operatingexpenses:Operatingincome':'oe_operating_income', 
                'Netincomepershare:Basic':'eps_basic2',
       }

In [ ]:
cols = list(rename_first.keys())
first = first[cols]
first = first.rename(columns=rename_first)

In [ ]:
assert first.shape[0] == first.index.nunique()

In [ ]:
first.info()

In [ ]:
first.eps_basic = first.eps_basic.combine_first(first.eps_basic2.rename('eps_basic'))
first = first.drop(columns=['eps_basic2'])

In [ ]:
final = pd.concat([second, first]).sort_values('year').groupby('year').first()
final

In [ ]:
final.net_income = final.net_income.combine_first(final.ie_net_income.rename('net_income'))
final = final.drop(columns=['ie_net_income'])

In [ ]:
df = final[second.columns]
df

In [ ]:
df.info()

In [ ]:
df.reset_index().to_csv('../assets/netflix_revenues.tsv', sep='\t', index=False)

Для создания датасета использовались таблицы `CONSOLIDATED STATEMENTS OF OPERATIONS` из официальных отчетов Netflix.

Структура этих таблиц менялась, поэтому в итоговой таблице есть пропущенные значения (часть информации из ранних таблиц не добавлена, часть столбцов вероятно можно объединить, если использовать знания экспертов).

С 2012 по 2023 год данные почти полные для выбранных столбцов.

Значения приводятся в тысячах долларов, за исключением значений на акцию (per share).

Структура таблицы из отчета и связь с датасетом:

|                                             | Название столбца в таблице отчета   |                            | название колонки                  | единицы измерения | не пропущенные значения с 2002 по 2023 | не пропущенные значения с 2012 по 2023 |
| ------------------------------------------- | ----------------------------------- | -------------------------- | --------------------------------- | ----------------- | -------------------------------------- | -------------------------------------- |
| Revenues:                                   |                                     |                            | revenues                          | thousands, `$`    | 22                                     | 12                                     |
|                                             | cost of revenues                    |                            | cost_of_revenues                  | thousands, `$`    | 20                                     | 12                                     |
|                                             |                                     | marketing                  | marketing                         | thousands, `$`    | 14                                     | 12                                     |
|                                             |                                     | technology and development | technology_and_development        | thousands, `$`    | 14                                     | 12                                     |
|                                             |                                     | general and administrative | general_and_administrative        | thousands, `$`    | 14                                     | 12                                     |
| operating income                            |                                     |                            | operating_income                  | thousands, `$`    | 14                                     | 12                                     |
| other income (expense):                     |                                     |                            |                                   |                   |                                        |                                        |
|                                             | interest expense                    |                            | interest_expense                  | thousands, `$`    | 17                                     | 12                                     |
|                                             | interest and other income (expense) |                            | interest_and_other_income_expense | thousands, `$`    | 12                                     | 12                                     |
| income before income taxes                  |                                     |                            | income_before_income_taxes        | thousands, `$`    | 20                                     | 12                                     |
| provision for (benefit from) income taxes   |                                     |                            | -                                 | thousands, `$`    |                                        |                                        |
| net income                                  |                                     |                            | net_income                        | thousands, `$`    | 12                                     | 12                                     |
| earnings per share:                         |                                     |                            |                                   |                   |                                        |                                        |
|                                             | basic                               |                            | eps_basic                         | `$`               | 20                                     | 12                                     |
|                                             | diluted                             |                            | eps_diluted                       | `$`               | 12                                     | 12                                     |
| Weighted-average common shares outstanding: |                                     |                            |                                   |                   |                                        |                                        |
|                                             | basic                               |                            | wacso_basic                       |                   | 21                                     | 11                                     |
|                                             | diluted                             |                            | wacso_diluted                     |                   | 11                                     | 11                                     |
|                                             |                                     |                            |                                   |                   |                                        |                                        |